## Preliminaries

In [1]:
import numpy as np
import pandas as pd
from math import *
import ast
import math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# load assortment data
assort = []
filename = open("assortment.txt","r")
for line in filename:
    assort.append(ast.literal_eval(line))

In [3]:
# load Choice Probability
prob = []
filename = open("probability.txt","r")
for line in filename:
    prob.append(np.array(ast.literal_eval(line)))

# Q1 

## Step 1: Get the beta coefficients using clean_train_long (0.8 of original training data, long format)

**Model 1: Multinomial Logit Model**

In [4]:
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

# Load the dataset
file_path = 'clean_train_long.csv'
data = pd.read_csv(file_path)

# Create a column for SKU based on the Product column, as explained previously
data['SKU'] = ((data['Product'] - 1) // 5) + 1
data.loc[data['Product'] == 0, 'SKU'] = 0

# Define the independent variables (features) for the model
X_reduced = data[['Cores', 'Frequency', 'TDP', 'Price']]
X_reduced = sm.add_constant(X_reduced)  # Add a constant to the model (intercept)

# The dependent variable is 'Chosen'
y = data['Chosen']

# Building the Multinomial Logit Model with the reduced set of features
mnl_model_reduced = sm.MNLogit(y, X_reduced).fit()

# Output the summary of the new model
model_summary = mnl_model_reduced.summary()

# Display the summary
model_summary

Optimization terminated successfully.
         Current function value: 0.329576
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:                 Chosen   No. Observations:                 7950
Model:                        MNLogit   Df Residuals:                     7945
Method:                           MLE   Df Model:                            4
Date:                Sun, 19 Nov 2023   Pseudo R-squ.:                  0.4157
Time:                        17:28:51   Log-Likelihood:                -2620.1
converged:                       True   LL-Null:                       -4484.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
==============================================================================
  Chosen=1       coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.9750      0.068    -28.936      0.000      -2.109      -1.841
Cores          1.0615      0.027     39.022      0.000       1.008       1.115
Frequency      0.7625      0.089      8.595      0.000       0.589       0.936
TDP           -0.0487      0.003    -17.672      0.000      -0.054      -0.043
Price         -0.0015      0.000    -14.889      0.000      -0.002      -0.001
==============================================================================
"""

**Model 2: biogeme**

In [5]:
import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta
from biogeme import models

# Load the data
file_path = 'clean_train_long.csv'
data = pd.read_csv(file_path)

# Create a BioGEME database from the data
database = db.Database('database', data)

# The following statement allows you to use the names of the variables as defined in the DataFrame.
globals().update(database.variables)

# Define the parameters to be estimated
B_Cores = Beta('B_Cores', 0, None, None, 0)
B_Frequency = Beta('B_Frequency', 0, None, None, 0)
B_TDP = Beta('B_TDP', 0, None, None, 0)
B_Price = Beta('B_Price', 0, None, None, 0)

# Define the utility function for the chosen alternative
V1 = B_Cores * Cores + B_Frequency * Frequency + B_TDP * TDP + B_Price * Price
# Utility of the non-chosen alternative is often normalized to 0.
V0 = 0

# Associate utility functions with the numbering of alternatives
V = {0: V0, 1: V1}

# Assume that both alternatives are available for all individuals
av = {0: 1, 1: 1}

# Define the model. Here, we use a binary logit model.
logprob = models.loglogit(V, av, Chosen)

# Create the Biogeme object
biogeme = bio.BIOGEME(database, logprob)
biogeme.modelName = 'Binary_Logit_Model'

# Estimate the model
results = biogeme.estimate()

# Get the results in a pandas DataFrame
pandas_results = results.getEstimatedParameters()
pandas_results

D:\anaconda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,Value,Rob. Std err,Rob. t-test,Rob. p-value
B_Cores,1.039711,0.023442,44.352411,0.000000
B_Frequency,0.185133,0.078382,2.361943,0.018179
B_Price,-0.001573,0.000104,-15.178099,0.000000
B_TDP,-0.049428,0.002318,-21.320253,0.000000


## Step 2: Use beta coefficients from previous step to obtain predicted probabilities for clean_test (0.2 of original training data)

In [6]:
df_validate = pd.read_csv('clean_test.csv')
#df_validate.head()
betas_1train = [-0.0015, 1.0615, 0.7625, -0.0487] # for sm.MNLogit method
betas_2train = [-0.001573, 1.039712, 0.185147, -0.049428] # for Biogeme method

**Model 1: Multinomial Logit Model**

In [7]:
def map_func(product_num):
    product_vec = list(range(4))
    
    if product_num != 0:
        payment_class = (product_num-1)%5
        if payment_class == 0:
            product_vec[0] = 3000
        elif payment_class == 1:
            product_vec[0] = 2700
        elif payment_class == 2:
            product_vec[0] = 2400
        elif payment_class == 3:
            product_vec[0] = 2100
        elif payment_class == 4:
            product_vec[0] = 1800

        sku = (product_num-1)//5
        if sku == 0:
            product_vec[1]=4; product_vec[2]=3.2; product_vec[3]=95
        elif sku == 1:
            product_vec[1]=8; product_vec[2]=2.9; product_vec[3]=60
        elif sku == 2:
            product_vec[1]=8; product_vec[2]=2.9; product_vec[3]=95
        elif sku == 3:
            product_vec[1]=4; product_vec[2]=2.9; product_vec[3]=60
        elif sku == 4:
            product_vec[1]=4; product_vec[2]=3.2; product_vec[3]=60
        elif sku == 5:
            product_vec[1]=4; product_vec[2]=2.2; product_vec[3]=135
    
#    product_vec = product_vec.reshape(-1,1)
    return product_vec #return (13x1)

In [8]:
def calculate_prob_within_assortment_2(betas_l, assortment, product):
    numerator = np.exp(np.dot(betas_l, product))
    denominator = sum(np.exp(np.dot(betas_l, map_func(assortment[j]))) for j in range(len(assortment)))
    return numerator/denominator

assortments = []
for k in df_validate['Assortment']:
    assortment = []
    k = ast.literal_eval(k)
    for i in k:
        assortment.append(calculate_prob_within_assortment_2(betas_1train, k, map_func(i)))
    assortments.append(np.array(assortment))

pred_prob = assortments

## Step 3: Extract true probability from clean_test (0.2 of original training data)

In [9]:
true_prob_pre = list(df_validate['Probability'])
true_prob = [np.array(ast.literal_eval(s)) for s in true_prob_pre]

## Step 4:Calculate RMSE

In [10]:
def cal_RMSE(true_prob, pred_prob): 
    # true_prob is the true probability, cal_prob is the predicted probability. They must have the same dimension.
    K=len(true_prob)
    if len(pred_prob)!=K:
        raise ValueError("Dimension mismatch")
    sum_error_sq=0
    total_item=0
    for a in range(K):
        total_item += len(true_prob[a])
        sum_error_sq += sum((true_prob[a]- pred_prob[a])**2)
    return np.sqrt(sum_error_sq/total_item)

In [11]:
print(f'RMSE for MNLogit: {cal_RMSE(true_prob, pred_prob)}')

RMSE for MNLogit: 0.20281620238735312


**Model 2: Biogeme**

Repeat Step 2 to 4 for Biogeme

In [17]:
def calculate_prob_within_assortment_2(betas_2, assortment, product):
    numerator = np.exp(np.dot(betas_2, product))
    denominator = sum(np.exp(np.dot(betas_2, map_func(assortment[j]))) for j in range(len(assortment)))
    return numerator/denominator

assortments = []
for k in df_validate['Assortment']:
    assortment = []
    k = ast.literal_eval(k)
    for i in k:
        assortment.append(calculate_prob_within_assortment_2(betas_2train, k, map_func(i)))
    assortments.append(np.array(assortment))

pred_prob2 = assortments

In [18]:
def cal_RMSE(true_prob, pred_prob): 
    # true_prob is the true probability, cal_prob is the predicted probability. They must have the same dimension.
    K=len(true_prob)
    if len(pred_prob)!=K:
        raise ValueError("Dimension mismatch")
    sum_error_sq=0
    total_item=0
    for a in range(K):
        total_item += len(true_prob[a])
        sum_error_sq += sum((true_prob[a]- pred_prob[a])**2)
    return np.sqrt(sum_error_sq/total_item)

In [19]:
print(f'RMSE for MNLogit: {cal_RMSE(true_prob, pred_prob2)}')

RMSE for MNLogit: 0.2453416215762017
